In [1]:
#Imports 
import cv2
import numpy as np
import math
import glob
import time
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from tqdm import tqdm
from feature_extraction import *
from PIL import Image, ImageEnhance

%matplotlib inline

In [10]:
# extract features from images
is_load = True
contours = []
labels   = []  # 1 for males  , 0 for females

if ( not is_load):
    # read male images
    flooded_path   = 'dataset_resized/flooded'
    flooded_files   = [ f for f in listdir(flooded_path) if isfile(join(flooded_path,f)) ]
    for i in tqdm(range(0, len(flooded_files))):
        img  = Image.open(join(flooded_path,flooded_files[i]))
        img, _  = preprocess_image( img )
        contour = get_contour_pixels(img)
        contours.append(contour)
        labels.append(1)
        
    ## read female images
    non_flooded_path = 'dataset_resized/non-flooded'
    non_flooded_files = [ f for f in listdir(non_flooded_path) if isfile(join(non_flooded_path,f)) ]
    for i in tqdm(range(0, len(non_flooded_files))):
        img  = Image.open( join(non_flooded_path,non_flooded_files[i]))
        img, _  = preprocess_image( img )
        contour = get_contour_pixels(img)
        contours.append(contour)
        labels.append(0)

    contours = np.asarray(contours , dtype= object)
    labels   = np.asarray(labels   , dtype= int )
    #save lables to dataset
    np.save('labels.npy', labels)

In [11]:
hinge_features = []
cold_features  = []
if(not is_load):
    for i in tqdm(range( len(contours)) ):
        feature  = get_hinge_features( contours[i] )
        hinge_features.append(feature)  
    hinge_features = np.asarray(hinge_features , dtype=object)
    np.save('features/hinge_features.npy', hinge_features)

    for i in tqdm(range( len(contours)) ):
        feature  = get_cold_features( contours[i] )
        cold_features.append(feature)  
    cold_features = np.asarray(cold_features , dtype=object)
    np.save('features/cold_features.npy', cold_features)

In [13]:
if(is_load):
    hinge_features = np.load('features/hinge_features.npy' , allow_pickle= True)
    cold_features = np.load('features/cold_features.npy' ,  allow_pickle= True)
    labels = np.load('features/labels.npy' , allow_pickle= True)

## concate features

In [14]:
# concatenate features in one flattened array
features = np.concatenate( (hinge_features, cold_features) , axis=1)
cold_features.shape

(922, 420)

In [15]:
# Split dataset into training set and test set
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split( hinge_features , labels , test_size=0.2 ,  random_state=109) # 80% training and 20% test
# X_train, X_test, y_train, y_test = train_test_split( hinge_features , labels , test_size=0.2 ,  random_state=50)
X_train, X_test, y_train, y_test = train_test_split( features , labels , test_size=0.2 ,  random_state=175)

In [22]:
from sklearn.preprocessing import StandardScaler

# Create a based model
scaler = StandardScaler()
X_train  = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


from sklearn.decomposition import PCA

pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


## SVM

In [23]:
#Import svm model
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report

#Create a svm Classifier
t0 = time.time()
SVM_clf = svm.SVC(kernel= "linear" ) # Linear Kernel

#Train the model using the training sets
SVM_clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = SVM_clf.predict(X_test)
t1 = time.time()
# clf.score(X_test, y_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Accuracy: 0.8486486486486486


## Random forest tree


In [24]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 50, 100],
    'min_samples_leaf': [1, 2,5],
    'min_samples_split': [ 2, 5],
    'n_estimators': [10,20, 100, 200]
}




# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(random_state=42), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

y_pred = grid_search.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 72 candidates, totalling 216 fits
{'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
0.8548531607765057
Accuracy: 0.8918918918918919


In [34]:
from sklearn.ensemble import GradientBoostingClassifier

# params for GradientBoostingClassifier
param_grid = {
    # 'max_depth': [10, 50, 100],
    # 'min_samples_leaf': [1, 2,5],
    # 'min_samples_split': [ 2, 5],
    'n_estimators': [ 200 , 500 , 1000]

}




# Instantiate the grid search model
grid_search =GridSearchCV(estimator =  GradientBoostingClassifier(random_state=42), param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)


y_pred = grid_search.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Accuracy: 0.8918918918918919


In [35]:
import pickle

# save the model to disk
filename = 'rfc.pkl'
pickle.dump(grid_search, open(filename, 'wb'))

